In [2]:
import imageio
import imgaug as ia
import os
import random
%matplotlib inline

# Load random image from dataset

In [3]:
dir_content = os.listdir("dataset/")
image_list = [img for img in dir_content if img.endswith(".png")]
file_list = [txt for txt in dir_content if txt.endswith(".txt")]
image = imageio.imread("dataset/{}".format(random.choice(image_list)))
# ia.imshow(image)

# Augment the image
`imgaug` can perform different techniques, first augmentation will be  `Affine`

In [4]:
from imgaug import augmenters as iaa
ia.seed(4)

rotate = iaa.Affine(rotate=(-25, 25))
image_aug = rotate(image=image)

print("Augmented with affine:")
# ia.imshow(image_aug)

Augmented with affine:


# Augment a batch of images
Changing the image param it's possible to achieve augmentation of N numbers of image, the example is
made with the same image different times.

* hstack() is used to combine the images into one larger for better display

In [5]:
import numpy as np

images = [image, image, image, image]
images_aug = rotate(images=images)

print("Augmented batch:")
# ia.imshow(np.hstack(images_aug))

Augmented batch:


# Use many augmentation techniques simultaneously

The next example we will combine several methods and apply them simultaneously to images. First, we instantiate
each technique on its own and apply them one after the other by calling `augmenter(images=...)` several times. Alternatively we can use `Sequential` to combine different augmenters into one pipeline and then apply them all in a single augmentation call. We will use `Affine`, some gaussian noise `AdditiveGaussianNoise` and crop the image randomly with `Crop`

In [ ]:
seq = iaa.Sequential([
    iaa.Affine(rotate=(-25, 25)),
    iaa.AdditiveGaussianNoise(scale=(5, 10)),
    iaa.Crop(percent=(0, 0.2))
])

images_aug = seq(images=images)

print("Sequential augmented:")
# ia.imshow(np.hstack(images_aug))

In [ ]:
# using sequential with random order
seq = iaa.Sequential([
    iaa.Affine(rotate=(-25, 25)),
    iaa.AdditiveGaussianNoise(scale=(5, 20)),
    iaa.Crop(percent=(0, 0.2))
], random_order=True)

images_aug = [seq(image=image) for _ in range(8)]

print("Random augmented:")
# ia.imshow(ia.draw_grid(images_aug, cols=4, rows=2))

# Augmenting images of different sizes
Load batch of different sizes, apply the augmentation as a single batch and show each image one by one with the input and output

In [ ]:
seq = iaa.Sequential([
    iaa.CropAndPad(percent=(-0.2, 0.2), pad_mode="edge"),#crop and pad img
    iaa.AddToHueAndSaturation((-50, 50)), #change color
    iaa.ElasticTransformation(alpha=90, sigma=9), #water effect
    iaa.Cutout() #replace one squared area within the image by a constant value
], random_order=True)

images_with_different_sizes = [imageio.imread("dataset/{}".format(img)) for img in random.sample(image_list, 1)]

#Augment them as one batch
images_aug = seq(images=images_with_different_sizes)

for idx, img in enumerate(images_aug):
    print("Image {} (input shape: {}, output shape: {})".format(idx, images_with_different_sizes[idx].shape, images_aug[idx].shape))
    ia.imshow(np.hstack([images_with_different_sizes[idx], img]))

# Dataset augmentation 

### Finished

yolo mark format: object-class x y width height

* `x` `y` `width` `height` are value within ranges 0-1 relative to the shape

In [18]:
import imageio
import imgaug as ia
import os
import random
import numpy as np
import cv2
%matplotlib inline

In [19]:
def read_coord(file, img):
    """
    Main function for reading the annotation file for the respective image.
    
    The yolo annotations are stored in a same name .txt file for the mirror .png image, inside the file
    the coordinates are stores as a class_name, x, y, width, height relative values of image. The first one
    is not used but the last four are stored in variables for later calculations.
    
    Parameters:
    
    file (file): The .txt file annotation
    img (ndarray): The image loaded
    
    Returns:
    
    bbs (imgaug object): The BoundingBoxesOnImage object containing all the coordinates annotated in the
    original .txt file.
    """
    with open("dataset/{}".format(file), "r") as file:
        temp_boundingbox_array = []
        for line in file.readlines():
            _, relative_x, relative_y, relative_width_box, relative_height_box = line.split(" ")
            print("x:{} y:{} HEIGHT:{} WIDTH:{}".format(relative_x, relative_y, \
                                                        relative_width_box, relative_height_box))

            center_y, center_x, real_width_box, real_height_box, y_1, x_1 = calculate_coord(img.shape, 
                                                                                            relative_x,
                                                                                          relative_y,
                                                                                          relative_width_box,
                                                                                          relative_height_box)

            bbs = BoundingBoxesOnImage([
                BoundingBox(x1=x_1, x2=(x_1+real_width_box), y1=y_1, y2=(y_1+real_height_box))
            ], shape=img.shape)
            x_2 = (x_1+real_width_box)
            y_2 = (y_1+real_height_box)
            temp_boundingbox_array.append([x_1, y_1, x_2, y_2])
        
    np_boundingbox = np.vstack(temp_boundingbox_array)
    bbs = BoundingBoxesOnImage.from_xyxy_array(np_boundingbox, img.shape)
    return bbs
        
def calculate_coord(shape, x, y, width, height):
    """
    Function responsible for calculating the coord values from yolo mark to top-left(x,y) position.
    
    Since the annotations in yolo format are based in relative positions, this function is responsible
    for converting them to the normal top-left format so it can be possible to work with imgaug library
    and his bounding boxes.
    
    Parameters:
    
    shape (tuple): The actual shape of the image which is being used
    x (float): Relative X value found in the yolo annotation
    y (float): Relative Y value found in the yolo annotation
    width (float): Relative WIDTH value found in the yolo annotation
    height (float): Relative HEIGHT value found in the yolo annoation
    
    Returns:
    
    abs_y: int value mapped of center Y of bounding box element
    abs_x: int value mapped of center X of bounding box element
    abs_w: int value mapped of the width size box
    abs_h: int value mapped of the height size box
    y_1: int value mapped of the top-left Y1
    x_2: int value mapped of the top-left X1
    """
    y_image = shape[0]
    x_image = shape[1]
    
    abs_w = int(float(width) * x_image)
    abs_h = int(float(height) * y_image)
    abs_y = int(float(y) * y_image)
    abs_x = int(float(x) * x_image)
    
    x_1 = int(abs_x - abs_w/2)
    y_1 = int(abs_y - abs_h/2)
    
    
    return abs_y, abs_x, abs_h, abs_w, y_1, x_1

def save_aug(file, img):
    """
    Save method for the new augmentated images.
    
    As new augments are generated for the dataset, this function is the one who will save the new pair of
    .png and .txt files to the disk.
    
    Parameters:
    
    file (file): The .txt file annotation
    img (ndarray): The image loaded
    
    Returns:
    
    None
    """
    new_image_name = "{:05d}.png".format(count)
    new_file_name = "{:05d}.txt".format(count)
    with open("dataset/{}".format(file), "r") as old_file:
        content = old_file.readlines()
    with open("dataset/{}".format(new_file_name), "w") as file:
        for line in content:
            file.write(line)
    cv2.imwrite("dataset/{}".format(new_image_name), img)
    print("Saving {} with {} annotation" .format(new_image_name, new_file_name))

In [ ]:
"""
Data augmentation in the dataset, 3 methods are chosen in the sequential constructor who will
be applied in random order to the batch loaded and saved in the desired folder.
"""

from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import imgaug.augmenters as iaa


seq = iaa.Sequential([
    iaa.AddToHueAndSaturation((-25, 50), per_channel=True),
    iaa.MultiplyHueAndSaturation((0.5, 1.5), per_channel=True),
    iaa.AddToBrightness((0, 50))
], random_order=True)
count = 0
random.shuffle(image_list)
for _ in range(5):
    for idx, img in enumerate(image_list):
        current_image = img
        current_annotation = current_image.replace(".png", ".txt")
        print("Image name: {}, respective annotation: {}\n\n" .format(current_image, current_annotation))
        image = imageio.imread("dataset/{}".format(current_image))
        bbs = read_coord(current_annotation, image)
        image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)
        save_aug(current_annotation, image_aug)
        count += 1
#         ia.imshow(np.hstack([bbs.draw_on_image(image, size=4), bbs_aug.draw_on_image(image_aug, size=3)]))